- There's a lot of data wrangling involved, so bear with me.
- I preview (with .head()) each dataframe once its created just to show where I'm going.
- I will be creating two interactive maps of Washington DC using the folium package.
- Map 1 will show the location of the top 20 most used bike stations for registered vs casual users, from the years 2011 to 2019.
- Map 2 will show a heat map of the most frequently used stations from the years 2011 to 2019.

In [1]:
import pandas as pd
import geopandas as gpd
import folium
from sklearn.preprocessing import minmax_scale
from collections import OrderedDict, defaultdict
from folium.plugins import HeatMapWithTime


<h3> 1) Some necessary data wrangling </h3>

<h4> 1.1 Load dataframes with stations data </h4>

In [2]:
# Import dataframes with stations data

stations_2011 = pd.read_csv('cleaned_data/stations_2011.csv')
stations_2012 = pd.read_csv('cleaned_data/stations_2012.csv')
stations_2013 = pd.read_csv('cleaned_data/stations_2013.csv')
stations_2014 = pd.read_csv('cleaned_data/stations_2014.csv')
stations_2015 = pd.read_csv('cleaned_data/stations_2015.csv')
stations_2016 = pd.read_csv('cleaned_data/stations_2016.csv')
stations_2017 = pd.read_csv('cleaned_data/stations_2017.csv')
stations_2018 = pd.read_csv('cleaned_data/stations_2018.csv')
stations_2019 = pd.read_csv('cleaned_data/stations_2019.csv')

df_2011 = pd.read_csv('cleaned_data/df_2011.csv')
df_2012 = pd.read_csv('cleaned_data/df_2012.csv')
df_2013 = pd.read_csv('cleaned_data/df_2013.csv')
df_2014 = pd.read_csv('cleaned_data/df_2014.csv')
df_2015 = pd.read_csv('cleaned_data/df_2015.csv')
df_2016 = pd.read_csv('cleaned_data/df_2016.csv')
df_2017 = pd.read_csv('cleaned_data/df_2017.csv')
df_2018 = pd.read_csv('cleaned_data/df_2018.csv')
df_2019 = pd.read_csv('cleaned_data/df_2019.csv')

- run time for future reference: 36 s 

<h4> 1.2) Preview of loaded dataframes </h4>

In [3]:
stations_2011.head()

,station,region,lat,lon
0,10th & Monroe St NE,"Washington, DC",38.932457,-76.993534
1,10th & U St NW,"Washington, DC",38.917200,-77.025900
2,10th St & Constitution Ave NW,"Washington, DC",38.893028,-77.026013
3,11th & F St NW,"Washington, DC",38.897857,-77.026975
4,11th & H St NE,0.0,0.000000,0.000000


In [4]:
df_2011.head()

,duration,start_date,end_date,start_station_number,start_station,end_station_number,end_station,bike_number,member_type,registered,casual
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,5th & F St NW,31620,5th & F St NW,W00247,Member,1,0
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,14th & Harvard St NW,31101,14th & V St NW,W00675,Casual,0,1
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,Georgia & New Hampshire Ave NW,31104,Adams Mill & Columbia Rd NW,W00357,Member,1,0
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,10th & U St NW,31503,Florida Ave & R St NW,W00970,Member,1,0
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,Adams Mill & Columbia Rd NW,31106,Calvert & Biltmore St NW,W00346,Casual,0,1


<h4> 1.3) Create new dataframes for folium </h3>

- Create dataframes **stations_full_20xx**
- for each year, we want to create a dataframe that includes: "station", "year", "region", "lat", "lon", "total", "registered", "casual"

In [5]:
df_all_years = [df_2011, df_2012, df_2013, df_2014, df_2015, df_2016, df_2017, df_2018, df_2019]
stations_all_years = [stations_2011, stations_2012, stations_2013, stations_2014, 
                        stations_2015, stations_2016, stations_2017, stations_2018, stations_2019]

stations_all_years_updated = []

for item, element in zip (df_all_years, stations_all_years):
    
    
    item['start_date'] = pd.to_datetime(item['start_date'])
    item['year'] = item['start_date'].dt.year

    start = item.groupby(['year', 'start_station']).agg({'start_station':'count', 'registered': 'sum', 'casual': 'sum' }).rename(columns = {'start_station':'start_total', 'registered': 'start_registered', 'casual':'start_casual'}) # "start" = station name + num users as start station
    start = start.sort_values(by = 'start_total', ascending=False)       # sort by ranking
    start.reset_index(inplace=True)                                                 


    end = item.groupby(['year', 'end_station']).agg({'end_station':'count', 'registered': 'sum', 'casual': 'sum' }).rename(columns = {'end_station':'end_total', 'registered': 'end_registered', 'casual':'end_casual'})    # "end" = station name + num users as end station
    end = end.sort_values(by = 'end_total', ascending=False)                                                            # sort by ranking
    end.reset_index(inplace=True)

    element = element.merge(start, how = 'left', left_on = 'station', right_on = 'start_station')                   # merge station_20xx with "start"
    element = element.merge(end, how = 'left', left_on = 'station', right_on = 'end_station')                       # merge station_20xx with "end"


    element.drop(['start_station', 'end_station'], axis = 1, inplace = True)

    element['total'] = element['start_total'] + element['end_total']
    element['registered'] = element ['start_registered'] + element ['end_registered']
    element['casual'] = element ['start_casual'] + element['end_casual']
    element = element[['year_x', 'station', 'region', 'lat', 'lon', 'total', 'registered','casual']].rename(columns = {'year_x':'year'})

    stations_all_years_updated.append(element)

# Store elements of "stations_all_years_updated" into variables
stations_full_2011 = stations_all_years_updated[0]
stations_full_2012 = stations_all_years_updated[1]
stations_full_2013 = stations_all_years_updated[2]
stations_full_2014 = stations_all_years_updated[3]
stations_full_2015 = stations_all_years_updated[4]
stations_full_2016 = stations_all_years_updated[5]
stations_full_2017 = stations_all_years_updated[6]
stations_full_2018 = stations_all_years_updated[7]
stations_full_2019 = stations_all_years_updated[8]


<h4> 1.4) Preview of dataframes stations_full_20xx </h4>

In [6]:
# Check dataframe 
stations_full_2011.head()


,year,station,region,lat,lon,total,registered,casual
0,2011,10th & Monroe St NE,"Washington, DC",38.932457,-76.993534,4384,3744,640
1,2011,10th & U St NW,"Washington, DC",38.917200,-77.025900,33984,30252,3732
2,2011,10th St & Constitution Ave NW,"Washington, DC",38.893028,-77.026013,35720,17972,17747
3,2011,11th & F St NW,"Washington, DC",38.897857,-77.026975,476,458,18
4,2011,11th & H St NE,0.0,0.000000,0.000000,490,470,20


<h4> 1.5) Create dataframes stations_reg_20xx and stations_cas_20xx </h4>

In [7]:
# Create separate dataframes for casual and registered users
# Sort by number of rides per station

stations_registered = []
stations_casual = []

stations_full = [stations_full_2011, stations_full_2012, stations_full_2013, stations_full_2014, 
                stations_full_2015, stations_full_2016, stations_full_2017, stations_full_2018, stations_full_2019]

for item in stations_full: 
    reg = item.copy(deep=True)[['station', 'region', 'lat', 'lon', 'registered']].sort_values(by = 'registered', ascending = False ).reset_index(drop=True)
    cas = item.copy(deep = True)[['station', 'region', 'lat', 'lon', 'casual']].sort_values(by = 'casual', ascending = False ).reset_index(drop=True)
    stations_registered.append(reg)
    stations_casual.append(cas)


# Assign colour to each station based on number of rides

num_years = len(stations_registered)

for i in range (num_years):
    stations_registered[i]['marker_colour'] = stations_registered[i]['registered'].apply(lambda x: '#D2042D' if x >= 50000  # red
                                                                            else '#FFEA00' if x >= 40000 # yellow
                                                                            else '#00FF00' if x >= 30000 # lime
                                                                            else '#0000FF' if x >= 20000  # blue
                                                                            else '#808080')                 # grey
    
    stations_casual[i]['marker_colour'] = stations_casual[i]['casual'].apply(lambda x: '#D2042D' if x >= 50000  # red
                                                                            else '#FFEA00' if x >= 40000 # yellow
                                                                            else '#00FF00' if x >= 30000 # lime
                                                                            else '#0000FF' if x >= 20000  # blue
                                                                            else '#808080')                 # grey


# Create separate dataframes for registered and casual users, by year
stations_reg_2011 = stations_registered[0]
stations_reg_2012 = stations_registered[1]
stations_reg_2013 = stations_registered[2]
stations_reg_2014 = stations_registered[3]
stations_reg_2015 = stations_registered[4]
stations_reg_2016 = stations_registered[5]
stations_reg_2017 = stations_registered[6]
stations_reg_2018 = stations_registered[7]
stations_reg_2019 = stations_registered[8]

stations_cas_2011 = stations_casual[0]
stations_cas_2012 = stations_casual[1]
stations_cas_2013 = stations_casual[2]
stations_cas_2014 = stations_casual[3]
stations_cas_2015 = stations_casual[4]
stations_cas_2016 = stations_casual[5]
stations_cas_2017 = stations_casual[6]
stations_cas_2018 = stations_casual[7]
stations_cas_2019 = stations_casual[8]


<h4> 1.6) Preview of dataframes stations_reg_20xx and stations_cas_20xx </h4>

In [8]:
stations_reg_2019.head()

,station,region,lat,lon,registered,marker_colour
0,Columbus Circle / Union Station,"Washington, DC",38.896960,-77.004930,113816.0,#D2042D
1,15th & P St NW,"Washington, DC",38.909801,-77.034427,70066.0,#D2042D
2,New Hampshire Ave & T St NW,"Washington, DC",38.915544,-77.038252,62954.0,#D2042D
3,1st & M St NE,"Washington, DC",38.905697,-77.005483,59851.0,#D2042D
4,Massachusetts Ave & Dupont Circle NW,"Washington, DC",38.910100,-77.044400,58263.0,#D2042D


In [9]:
stations_cas_2019.head()

,station,region,lat,lon,casual,marker_colour
0,Lincoln Memorial,"Washington, DC",38.888255,-77.049436,35077.0,#00FF00
1,Henry Bacon Dr & Lincoln Memorial Circle NW,"Washington, DC",38.890539,-77.049383,32227.0,#00FF00
2,Jefferson Dr & 14th St SW,"Washington, DC",38.888553,-77.032427,29955.0,#0000FF
3,4th St & Madison Dr NW,"Washington, DC",38.890496,-77.017246,29472.0,#0000FF
4,Smithsonian-National Mall / Jefferson Dr & 12t...,"Washington, DC",38.888774,-77.028694,29193.0,#0000FF


<h3> 2) Interactive map showing top 20 stations </h3>

<h4> 2.1) Prepare Washington DC zoning map </h4>

- I downloaded the zoning map from the Open Data DC website
- Link: https://opendata.dc.gov/datasets/DCGIS::zoning-regulations-of-2016/explore?location=38.905967%2C-77.035882%2C12.72

In [10]:
# Load the geojson file, containing all the zones in Washington DC
# Use geopandas
zoning = gpd.read_file("shp_files/Zoning_Regulations_of_2016.geojson")

In [11]:
zoning.head()

,ZONING,ZONING_WEB_URL,ZONING_CHANGE_NARRATIVE,ZONING_STATUS,ZONING_LABEL,ZONE_DESCRIPTION,ZONE_DISTRICT,IZ_DESIGNATION,ZR58,ZR16,GLOBALID,OBJECTID,SHAPEAREA,SHAPELEN,geometry
0,MU-7B,https://handbook.dcoz.dc.gov/zones/mixed-use/M...,"Fixed map error of SSL 4507 936, 07/14/11, Mis...",Final,MU-7B,Permits medium density mixed-use development,Mixed-Use Zone,None,C-3-A,MU-7B,{CA941084-283B-4ADB-BDB4-2F5F4D5F8C33},515010,0,0,"POLYGON ((-76.98059 38.90251, -76.98070 38.902..."
1,MU-4,https://handbook.dcoz.dc.gov/zones/mixed-use/M...,"Official Digital Zoning Map of July 1, 2010",Final,MU-4,Permits moderate density mixed use development,Mixed-Use Zone,None,C-2-A,MU-4,{EE555AF3-9EAA-4FBB-997A-CA12025CBCD6},515011,0,0,"POLYGON ((-76.99314 38.90401, -76.99304 38.903..."
2,RA-4,https://handbook.dcoz.dc.gov/zones/residential...,"Official Digital Zoning Map of July 1, 2010",Final,RA-4,Permits medium to high-desnity apartments,Residential Apartment Zone,None,R-5-D,RA-4,{13788974-A5B4-4AC9-94B4-801535FFBA0A},515012,0,0,"POLYGON ((-77.00907 38.90328, -77.00907 38.903..."
3,MU-4,https://handbook.dcoz.dc.gov/zones/mixed-use/M...,"Official Digital Zoning Map of July 1, 2010",Final,MU-4,Permits moderate density mixed use development,Mixed-Use Zone,None,C-2-A,MU-4,{771870F8-B2D8-4608-9387-BE720940AEC8},515013,0,0,"POLYGON ((-77.01277 38.97397, -77.01375 38.974..."
4,MU-3A,https://handbook.dcoz.dc.gov/zones/mixed-use/M...,"Official Digital Zoning Map of July 1, 2010",Final,MU-3A,Permits low density mixed use development,Mixed-Use Zone,None,C-1,MU-3A,{2A223A8F-C12B-452B-90F2-367FAC4AFA25},515014,0,0,"POLYGON ((-77.00107 38.94075, -77.00107 38.940..."


In [12]:
# Drop unnecessary columns
zoning.drop(['ZONING_WEB_URL', 'ZONING_CHANGE_NARRATIVE', 'ZONING_STATUS', 
            'ZONING_LABEL', 'ZONE_DESCRIPTION', 'IZ_DESIGNATION','OBJECTID', 'SHAPEAREA'], 
            axis = 1, inplace = True)

In [13]:
# Check and count number of zones
print (zoning['ZONE_DISTRICT'].unique()) 
print (len(zoning['ZONE_DISTRICT'].unique()))

['Mixed-Use Zone' 'Residential Apartment Zone' 'Downtown Zone'
 'Production, Distribution, and Repair Zone' 'Residential Zone' 'Unzoned'
 'Special Purpose Zone' 'Neighborhood Mixed-Use Zone'
 'Residential Flat Zone']
9


In [14]:
# There are nine types of zones
# We want to combine them and colour-code them as follows:
# 0: mixed-use
# 1: residential
# 2: unzoned
# 3: production, distribution, repair
# 4: downtown
# 5: special purpose

zoning_colour_dict = {'Mixed-Use Zone': 0,
                    'Neighborhood Mixed-Use Zone': 0,
                    'Residential Apartment Zone': 1,
                    'Residential Zone': 1,
                    'Residential Flat Zone': 1,
                    'Downtown Zone': 4,
                    'Production, Distribution, and Repair Zone': 3,
                    'Unzoned': 2,
                    'Special Purpose Zone': 5}

zoning['zone_district_colour'] = zoning['ZONE_DISTRICT'].map(zoning_colour_dict)

In [15]:
zoning.head()

,ZONING,ZONE_DISTRICT,ZR58,ZR16,GLOBALID,SHAPELEN,geometry,zone_district_colour
0,MU-7B,Mixed-Use Zone,C-3-A,MU-7B,{CA941084-283B-4ADB-BDB4-2F5F4D5F8C33},0,"POLYGON ((-76.98059 38.90251, -76.98070 38.902...",0
1,MU-4,Mixed-Use Zone,C-2-A,MU-4,{EE555AF3-9EAA-4FBB-997A-CA12025CBCD6},0,"POLYGON ((-76.99314 38.90401, -76.99304 38.903...",0
2,RA-4,Residential Apartment Zone,R-5-D,RA-4,{13788974-A5B4-4AC9-94B4-801535FFBA0A},0,"POLYGON ((-77.00907 38.90328, -77.00907 38.903...",1
3,MU-4,Mixed-Use Zone,C-2-A,MU-4,{771870F8-B2D8-4608-9387-BE720940AEC8},0,"POLYGON ((-77.01277 38.97397, -77.01375 38.974...",0
4,MU-3A,Mixed-Use Zone,C-1,MU-3A,{2A223A8F-C12B-452B-90F2-367FAC4AFA25},0,"POLYGON ((-77.00107 38.94075, -77.00107 38.940...",0


<h4> 2.2) Create interactive map with folium </h4>

- Top 20 stations by registered vs casual users, sorted by year from 2011 to 2019

In [16]:
# Create base map
f = folium.Figure(width=1000, height=600)
m = folium.Map(control_scale=True, location=[38.900497,-77.007507],pointer_events=True, zoom_start = 13)

N = 20

# List of dataframes with top N start stations for registered and casual users
reg_all_years = [df.loc[:N, :] for df in stations_registered]
cas_all_years = [df.loc[:N, :] for df in stations_casual]


# List of all years
years_list = ['2011', '2012', '2013','2014', '2015', '2016', '2017', '2018', '2019' ]

# Create a list of FeatureGroup instances for all years
feature_groups_reg = [folium.FeatureGroup(name = f"{year} top 20 stations (registered)", show = False) for year in years_list]
feature_groups_cas = [folium.FeatureGroup(name = f"{year} top 20  stations (casual)", show = False) for year in years_list]

# Add child for each FeatureGroup instance
for fg in feature_groups_reg:
    m.add_child(fg)

for fg in feature_groups_cas:
    m.add_child(fg)


# Add circlemarkers to each FeatureGroup instance
for year in range (len(reg_all_years)):
    for index, row in reg_all_years[year].iterrows():
        
        # Create tool-tip label showing name of station and number of rides
        label = f"Station: {row['station']} /// num. rides: {row['registered']}"
        label = folium.Tooltip(label)
        folium.CircleMarker([row['lat'], row['lon']], radius = 10, tooltip = label, 
                            color = row['marker_colour'], fill = True, 
                            fill_color = row['marker_colour']).add_to(feature_groups_reg[year])

for year in range (len(cas_all_years)):
    for index, row in cas_all_years[year].iterrows():
        
        # Create tool-tip label
        label = f"Station:{row['station']} /// num. rides: {row['casual']}"
        label = folium.Tooltip(label)
        folium.CircleMarker([row['lat'], row['lon']], radius = 10, tooltip = label, 
                            color = row['marker_colour'], fill = True, 
                            fill_color = row['marker_colour']).add_to(feature_groups_cas[year])

# Create dataframe with just "ZONING" and "zone_district_colour" columns
zoning_just_df = zoning[['ZONING', 'zone_district_colour']]

# Add zoning map layer
folium.Choropleth(geo_data = zoning, data = zoning_just_df, columns = ['ZONING', 'zone_district_colour'],
                    key_on='feature.properties.ZONING', fill_opacity=0.3,
                    fill_color='Set1', legend_name='mixed-use  |  residential    |    unzoned    |    production     |     downtown     |     special').add_to(m)


# Add every FeatureGroup instance to base map "m"
for fg in feature_groups_reg:
    fg.add_to(m)

for fg in feature_groups_cas:
    fg.add_to(m)

# Enable LayerControl
folium.LayerControl().add_to(m)    
f.add_child(m)
f

outfp = "top_20_stations.html"
f.save(outfp)

- To view the interactive map, please follow this link: http://nbviewer.org/github/kyungoh22/bike_analysis/blob/main/interactive_maps/top_20_stations.html

<h3> 3) Interactive heatmap of stations </h3>

- Heatmap with time based on total rides per station from years 2011 to 2019

<h4> 3.1) Preparing the data </h4>

In [17]:
df_all_years = pd.concat(stations_all_years_updated)
df_all_years 

,year,station,region,lat,lon,total,registered,casual
0,2011,10th & Monroe St NE,"Washington, DC",38.932457,-76.993534,4384.0,3744.0,640.0
1,2011,10th & U St NW,"Washington, DC",38.917200,-77.025900,33984.0,30252.0,3732.0
2,2011,10th St & Constitution Ave NW,"Washington, DC",38.893028,-77.026013,35720.0,17972.0,17747.0
3,2011,11th & F St NW,"Washington, DC",38.897857,-77.026975,476.0,458.0,18.0
4,2011,11th & H St NE,0.0,0.000000,0.000000,490.0,470.0,20.0
...,...,...,...,...,...,...,...,...
646,2019,Wisconsin Ave & O St NW,"Washington, DC",38.908490,-77.063586,18020.0,15921.0,2099.0
647,2019,Woodglen Dr & Executive Blvd,"Montgomery County, MD (North)",39.043170,-77.113500,1558.0,1236.0,322.0
648,2019,Woodley Park Metro / Calvert St & Connecticut ...,"Washington, DC",38.923389,-77.051833,23236.0,19578.0,3658.0
649,2019,Woodmont Ave & Strathmore St,"Montgomery County, MD (South)",38.979875,-77.093522,5261.0,4301.0,960.0


- Folium's HeatMapWithTime() function requires us to pass in normalised values between 0 and 1

In [18]:
# use sklearn's minmax_scale to normalize to range (0-1)
df_all_years ['total'] = minmax_scale(df_all_years['total'])




- Make sure to remove all NaN values, otherwise folium's HeatMapWithTime won't work

In [19]:
# Make sure to remove all NaN values, otherwise HeatMapWithTime won't work
df_all_years.dropna(inplace = True)

- Check dataframe

In [20]:
df_all_years[['year', 'lat', 'lon', 'total']]

,year,lat,lon,total
0,2011,38.932457,-76.993534,0.030509
1,2011,38.917200,-77.025900,0.236596
2,2011,38.893028,-77.026013,0.248682
3,2011,38.897857,-77.026975,0.003300
4,2011,0.000000,0.000000,0.003398
...,...,...,...,...
646,2019,38.908490,-77.063586,0.125448
647,2019,39.043170,-77.113500,0.010833
648,2019,38.923389,-77.051833,0.161764
649,2019,38.979875,-77.093522,0.036615


- Finally create an OrderedDict with the year as the key and the "lat", "lon", "total" as values

In [21]:
# initialise defaultdict
# doesn't throw key error when you try to access missing key
heatmap_dict = defaultdict(list)

for row in df_all_years.itertuples():
    heatmap_dict[row.year].append([row.lat, row.lon, row.total])

# convert into OrderedDict to preserve order of years
heatmap_dict = OrderedDict(sorted(heatmap_dict.items()))


<h4> 3.2) Create interactive heatmap using folium </h4>

In [22]:
# Base map
m=folium.Map(location=[38.900497,-77.007507],zoom_start=12,min_zoom=11,zoom_control=False,Control_scale=True)

# heat map with time
hm = HeatMapWithTime(data=list(heatmap_dict.values()),
                     index=list(heatmap_dict.keys()), 
                     radius=20,
                     auto_play=False,
                     max_opacity=0.8)
# add heatmap to base map
hm.add_to(m)
m

m.save('heat_map_with_time.html')

- To view the interactive map, please follow this link: http://nbviewer.org/github/kyungoh22/bike_analysis/blob/main/interactive_maps/heat_map_with_time.html